In [3]:
import sqlalchemy
from sqlalchemy import create_engine
import os
from dotenv import load_dotenv


In [6]:
load_dotenv()
user = os.getenv('user')
password = os.getenv('password')
host = os.getenv('host')
db = os.getenv('db')
engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:5439/{db}')

In [7]:
### using in memory db
engine = create_engine('sqlite:///:memory:', echo=True)

In [8]:
### declaring a mapping 
from sqlalchemy.orm import declarative_base
Base = declarative_base()

In [9]:
## creating a mapped class for a single table
from sqlalchemy import Column, Integer, String
class User(Base): # inherits from declarative base
    __tablename__ = 'users'
    # defining the table properties and attributes -> primary_key column is necessary

    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    nickname = Column(String)


    def __repr__(self) -> str: # defines the format of our output
        return "<User(name='%s', fullname='%s', nickname='%s')>" % (
            self.name,
            self.fullname,
            self.nickname
        )

In [11]:
## inspecting our table metadata
User.__table__

Table('users', MetaData(), Column('id', Integer(), table=<users>, primary_key=True, nullable=False), Column('name', String(), table=<users>), Column('fullname', String(), table=<users>), Column('nickname', String(), table=<users>), schema=None)

In [13]:
# using the Metadata registry to create a table
Base.metadata.create_all(engine)

2023-03-17 21:41:32,586 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-17 21:41:32,595 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("users")
2023-03-17 21:41:32,597 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-17 21:41:32,603 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("users")
2023-03-17 21:41:32,604 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-03-17 21:41:32,610 INFO sqlalchemy.engine.Engine 
CREATE TABLE users (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	fullname VARCHAR, 
	nickname VARCHAR, 
	PRIMARY KEY (id)
)


2023-03-17 21:41:32,613 INFO sqlalchemy.engine.Engine [no key 0.00233s] ()
2023-03-17 21:41:32,615 INFO sqlalchemy.engine.Engine COMMIT


In [ ]:
## for firebird and Oracle sequence is required to generate new PK

from sqlalchemy import Sequence

Column(Integer, Sequence("user_id_seq"), primary_key=True)

class User(Base):
    __tablename__ = "users"
    id = Column(Integer, Sequence("user_id_seq"), primary_key=True)
    name = Column(String(50))
    fullname = Column(String(50))
    nickname = Column(String(50))

    def __repr__(self):
        return "<User(name='%s', fullname='%s', nickname='%s')>" % (
            self.name,
            self.fullname,
            self.nickname,
        )


In [14]:
# creating instance of the mapped class User
ed_user = User(name='ed', fullname='Éd Jones', nickname= 'edsnickname')
ed_user.name

'ed'

Note the Base declarative system automatically provided the __init__() constructor class this can be declared explicitly which will over ride the default provided by Base

In [15]:
# creating a session this allows us to speak to the db
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)


In [16]:
session = Session()

In [17]:
# adding objects to our User
ed_user = User(name='ed', fullname='Éd Jones', nickname= 'edsnickname')
session.add(ed_user)

In [18]:
# Getting our user
our_user = (
    session.query(User).filter_by(name='ed').first()
)

2023-03-17 21:57:28,960 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-17 21:57:28,973 INFO sqlalchemy.engine.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2023-03-17 21:57:28,974 INFO sqlalchemy.engine.Engine [generated in 0.00183s] ('ed', 'Éd Jones', 'edsnickname')
2023-03-17 21:57:28,993 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name = ?
 LIMIT ? OFFSET ?
2023-03-17 21:57:28,994 INFO sqlalchemy.engine.Engine [generated in 0.00091s] ('ed', 1, 0)


In [19]:
our_user

<User(name='ed', fullname='Éd Jones', nickname='edsnickname')>

In [20]:
ed_user is our_user

True

In [21]:
# use add_all() to add more than one User object at once

session.add_all(
    [
        User(name="wendy", fullname="Wendy Williams", nickname="windy"),
        User(name="mary", fullname="Mary Contrary", nickname="mary"),
        User(name="fred", fullname="Fred Flintstone", nickname="freddy"),
    ]
)

In [22]:
# we can also change objects attribute simply by reassigning them

ed_user.nickname = 'eddie'

In [23]:
session.dirty

IdentitySet([<User(name='ed', fullname='Éd Jones', nickname='eddie')>])

In [24]:
session.new # displays pending commits to be made

IdentitySet([<User(name='wendy', fullname='Wendy Williams', nickname='windy')>, <User(name='mary', fullname='Mary Contrary', nickname='mary')>, <User(name='fred', fullname='Fred Flintstone', nickname='freddy')>])

In [25]:
# to add changes to the db we need to commit

session.commit()

2023-03-17 22:27:14,387 INFO sqlalchemy.engine.Engine UPDATE users SET nickname=? WHERE users.id = ?
2023-03-17 22:27:14,395 INFO sqlalchemy.engine.Engine [generated in 0.01146s] ('eddie', 1)
2023-03-17 22:27:14,411 INFO sqlalchemy.engine.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?), (?, ?, ?), (?, ?, ?) RETURNING id
2023-03-17 22:27:14,412 INFO sqlalchemy.engine.Engine [generated in 0.00036s (insertmanyvalues)] ('wendy', 'Wendy Williams', 'windy', 'mary', 'Mary Contrary', 'mary', 'fred', 'Fred Flintstone', 'freddy')
2023-03-17 22:27:14,419 INFO sqlalchemy.engine.Engine COMMIT


In [26]:
ed_user.id

2023-03-17 22:44:26,272 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-17 22:44:26,290 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.id = ?
2023-03-17 22:44:26,291 INFO sqlalchemy.engine.Engine [generated in 0.00424s] (1,)


1

In [27]:
# Rolling back Transactions

ed_user.name = 'Edwardo'

In [28]:
fake_user = User(name='Fakeuser', fullname='Invalid', nickname='123')
session.add(fake_user)

In [29]:
session.new

IdentitySet([<User(name='Fakeuser', fullname='Invalid', nickname='123')>])

In [30]:
session.query(User).filter(User.name.in_(['Edwardo', 'Fakeuser'])).all()

2023-03-17 22:56:14,141 INFO sqlalchemy.engine.Engine UPDATE users SET name=? WHERE users.id = ?
2023-03-17 22:56:14,143 INFO sqlalchemy.engine.Engine [generated in 0.00195s] ('Edwardo', 1)
2023-03-17 22:56:14,156 INFO sqlalchemy.engine.Engine INSERT INTO users (name, fullname, nickname) VALUES (?, ?, ?)
2023-03-17 22:56:14,158 INFO sqlalchemy.engine.Engine [cached since 3525s ago] ('Fakeuser', 'Invalid', '123')
2023-03-17 22:56:14,162 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name IN (?, ?)
2023-03-17 22:56:14,163 INFO sqlalchemy.engine.Engine [generated in 0.00113s] ('Edwardo', 'Fakeuser')


[<User(name='Edwardo', fullname='Éd Jones', nickname='eddie')>,
 <User(name='Fakeuser', fullname='Invalid', nickname='123')>]

In [31]:
# if we roll back the transaction fakeuser will be kicked out and ed_username will be set back to ed

session.rollback()

2023-03-17 22:57:24,044 INFO sqlalchemy.engine.Engine ROLLBACK


In [32]:
ed_user.name

2023-03-17 22:57:29,868 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-03-17 22:57:29,869 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.id = ?
2023-03-17 22:57:29,869 INFO sqlalchemy.engine.Engine [cached since 783.6s ago] (1,)


'ed'

In [33]:
fake_user in session

False

In [34]:
session.query(User).filter(User.name.in_(['ed', 'Fakeuser'])).all()

2023-03-17 22:58:55,339 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name IN (?, ?)
2023-03-17 22:58:55,341 INFO sqlalchemy.engine.Engine [cached since 161.2s ago] ('ed', 'Fakeuser')


[<User(name='ed', fullname='Éd Jones', nickname='eddie')>]

##### QUERYING 

we can query the session for instances in our User 

In [35]:
for instance in session.query(User).order_by(User.id):
    print(instance.id, instance.fullname, instance.nickname)

2023-03-17 23:02:16,024 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users ORDER BY users.id
2023-03-17 23:02:16,028 INFO sqlalchemy.engine.Engine [generated in 0.00449s] ()
1 Éd Jones eddie
2 Wendy Williams windy
3 Mary Contrary mary
4 Fred Flintstone freddy


In [37]:
for name, fullname in session.query(User.name, User.fullname):
    print(name, fullname)

2023-03-17 23:18:07,646 INFO sqlalchemy.engine.Engine SELECT users.name AS users_name, users.fullname AS users_fullname 
FROM users
2023-03-17 23:18:07,647 INFO sqlalchemy.engine.Engine [generated in 0.00276s] ()
ed Éd Jones
wendy Wendy Williams
mary Mary Contrary
fred Fred Flintstone


In [38]:
for row in session.query(User, User.name).all():
    print(row.User, row.name)

2023-03-17 23:19:32,762 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname, users.name AS users_name__1 
FROM users
2023-03-17 23:19:32,765 INFO sqlalchemy.engine.Engine [generated in 0.00298s] ()
<User(name='ed', fullname='Éd Jones', nickname='eddie')> ed
<User(name='wendy', fullname='Wendy Williams', nickname='windy')> wendy
<User(name='mary', fullname='Mary Contrary', nickname='mary')> mary
<User(name='fred', fullname='Fred Flintstone', nickname='freddy')> fred


In [39]:
# using label to control individual column expression

for row in session.query(User.name.label('name_label')).all():
    print(row.name_label)

2023-03-17 23:21:42,276 INFO sqlalchemy.engine.Engine SELECT users.name AS name_label 
FROM users
2023-03-17 23:21:42,279 INFO sqlalchemy.engine.Engine [generated in 0.00173s] ()
ed
wendy
mary
fred


In [40]:
# using aliased table name
from sqlalchemy.orm import aliased
user_alias = aliased(User, name='user_alias')

for row in session.query(user_alias, user_alias.name).all():
    print(row.user_alias)

2023-03-17 23:37:14,142 INFO sqlalchemy.engine.Engine SELECT user_alias.id AS user_alias_id, user_alias.name AS user_alias_name, user_alias.fullname AS user_alias_fullname, user_alias.nickname AS user_alias_nickname, user_alias.name AS user_alias_name__1 
FROM users AS user_alias
2023-03-17 23:37:14,144 INFO sqlalchemy.engine.Engine [generated in 0.00236s] ()
<User(name='ed', fullname='Éd Jones', nickname='eddie')>
<User(name='wendy', fullname='Wendy Williams', nickname='windy')>
<User(name='mary', fullname='Mary Contrary', nickname='mary')>
<User(name='fred', fullname='Fred Flintstone', nickname='freddy')>


In [41]:
# offset and limit using python array slices
for u in session.query(User).order_by(User.id)[1:3]: # start from second row offset 1
    print(u)

2023-03-17 23:40:06,706 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users ORDER BY users.id
 LIMIT ? OFFSET ?
2023-03-17 23:40:06,708 INFO sqlalchemy.engine.Engine [generated in 0.00178s] (2, 1)
<User(name='wendy', fullname='Wendy Williams', nickname='windy')>
<User(name='mary', fullname='Mary Contrary', nickname='mary')>


In [42]:
## using filter() or filter_by(keyword=)

for (name, ) in session.query(User.name).filter_by(fullname='Ed Jones'):
    print(name)

2023-03-18 00:08:14,114 INFO sqlalchemy.engine.Engine SELECT users.name AS users_name 
FROM users 
WHERE users.fullname = ?
2023-03-18 00:08:14,115 INFO sqlalchemy.engine.Engine [generated in 0.00100s] ('Ed Jones',)


In [43]:
for (name, ) in session.query(User.name).filter(User.fullname == 'Ed Jones'):
    print(name)

2023-03-18 00:09:13,739 INFO sqlalchemy.engine.Engine SELECT users.name AS users_name 
FROM users 
WHERE users.fullname = ?
2023-03-18 00:09:13,740 INFO sqlalchemy.engine.Engine [cached since 59.63s ago] ('Ed Jones',)


In [47]:
## using multiple filters like multiple AND conditions
for user in (
    session.query(User).filter(User.name == 'ed').filter(User.fullname == 'Éd Jones')
):
    print(user)

2023-03-18 00:13:38,594 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name = ? AND users.fullname = ?
2023-03-18 00:13:38,595 INFO sqlalchemy.engine.Engine [cached since 119.8s ago] ('ed', 'Éd Jones')
<User(name='ed', fullname='Éd Jones', nickname='eddie')>


#### useful column operators

query.filter(User.name != "ed")

query.filter(User.name.like('%ed%')) -> case sensitive

query.filter(User.name.ilike('%ed%')) -> case insensitive

query.filter(User.name == 'ed')

query.filter(User.name.in_(['ed', 'wendy']))

query.filter(~User.name.in_(["ed", "wendy", "jack"]))

query.filter(User.name == None)

query.filter(User.name.is_(None)) -> alternatively, if pep8/linters are a concern

query.filter(User.name != None)


query.filter(User.name.is_not(None)) -> alternatively, if pep8/linters are a concern



In [48]:
for user in session.query(User).filter(User.name.in_(
    session.query(User.name).filter(User.name.ilike('%ed%'))
)):
    print(user)

2023-03-18 00:26:30,882 INFO sqlalchemy.engine.Engine SELECT users.id AS users_id, users.name AS users_name, users.fullname AS users_fullname, users.nickname AS users_nickname 
FROM users 
WHERE users.name IN (SELECT users.name 
FROM users 
WHERE lower(users.name) LIKE lower(?))
2023-03-18 00:26:30,886 INFO sqlalchemy.engine.Engine [generated in 0.00476s] ('%ed%',)
<User(name='ed', fullname='Éd Jones', nickname='eddie')>
<User(name='fred', fullname='Fred Flintstone', nickname='freddy')>


In [ ]:
# use and_()
from sqlalchemy import and_
query.filter(and_(User.name == 'ed', User.fullname == 'Ed Jones'))

# or send multiple expressions to .filter()
query.filter(User.name == 'ed', User.fullname == 'Ed Jones')

# or chain multiple filter()/filter_by() calls
query.filter(User.name == 'ed').filter(User.fullname == 'Ed Jones')

In [ ]:
from sqlalchemy import or_
query.filter(or_(User.name == 'ed', User.name == 'wendy'))